In [2]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import datetime
import os
import glob
import numpy as np
import pandas as pd
import subprocess
import xarray as xr

In [3]:
dirs_to_make = ['Data/RAW', 'Data/PROCCED_hourly']
for directory in dirs_to_make:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [12]:
start_date = datetime.datetime(2022,12,23,0)
end_date = datetime.datetime(2023,1,17,0)

In [5]:
start_date

datetime.datetime(2022, 12, 23, 0, 0)

In [13]:
## Connect to AWS s3
s3_resource = boto3.resource("s3", config=Config(signature_version=UNSIGNED))
s3_bucket   = "noaa-mrms-pds"

In [14]:
def download_MRMS_AWS(yyyy,mm,dd,hh):
    s3_file_path = f'CONUS/MultiSensor_QPE_01H_Pass2_00.00/{yyyy}{mm}{dd}'
    s3_object_name = f'MRMS_MultiSensor_QPE_01H_Pass2_00.00_{yyyy}{mm}{dd}-{hh}0000.grib2.gz'
    if not os.path.exists(f'./Data/RAW/{s3_object_name}') and not os.path.exists(f'./Data/RAW/{s3_object_name[:-3]}'):
        s3_resource.Bucket(s3_bucket).download_file(
            Filename=f'./Data/RAW/{s3_object_name}',
            Key=f'{s3_file_path}/{s3_object_name}')

In [15]:
target_time = start_date
while target_time < end_date:
    yyyy = target_time.strftime("%Y")
    mm = target_time.strftime("%m")
    dd = target_time.strftime("%d")
    hh = target_time.strftime("%H")
    download_MRMS_AWS(yyyy,mm,dd,hh)
    target_time += datetime.timedelta(hours=1)
    # print(target_time)

## Unzip .gz:

In [16]:
files = glob.glob('Data/RAW/*.gz')
files.sort()

In [17]:
def unzip_gz_convert_grib(filename):
    if not filename.endswith('.gz'):
        raise Exception("The filename should end with .gz")
    bash_command = f'gunzip {filename}'
    process = subprocess.Popen(bash_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()    

In [18]:
for filename in files:
    unzip_gz_convert_grib(filename)

## Extract data:

In [2]:
files = glob.glob('Data/RAW/*.grib2')
files.sort()

In [3]:
filename = files[0]

In [4]:
ds = xr.open_dataset(filename, engine='cfgrib')

ValueError: unrecognized engine cfgrib must be one of: ['scipy', 'store']

In [ ]:
!pip install cfgrib

In [1]:
import cfgrib

ModuleNotFoundError: No module named 'cfgrib'